In [ ]:
# import pdb; pdb.set_trace()
# import pudb; pudb.set_trace()

# df.style.background_gradient(
#     cmap='coolwarm', 
#     axis=None, 
#     vmin=-1, 
#     vmax=1
# )  # 'Blues', 'coolwarm', 'RdBu_r' & 'BrBG'

# pd.set_option('precision', 2)

# .plot_bokeh(sizing_mode="scale_height")
# .plot_bokeh(kind='hist', bins=1000, sizing_mode="scale_height", xlim=[0, .1])

# Imports

In [75]:
%load_ext autoreload
%autoreload 2

import pandas as pd
pd.set_option('float_format', '{:f}'.format)
pd.set_option('display.max_rows', 40)
pd.set_option('display.max_columns', 99)

import matplotlib as mpl
from matplotlib import style
style.use('fivethirtyeight')
mpl.rcParams['figure.figsize'] = [20, 7]

import datetime as dt
import pickle
import numpy as np
import pandas as pd
import pandas_bokeh
pandas_bokeh.output_file("tmp/bokeh_output.html")
import ray

from data_layer import arrow_dataset, storage_adaptor
from tick_filter import streaming_tick_filter
from tick_sampler import streaming_tick_sampler, daily_stats
from workflows import sampler_task, sampler_flow, syncbar_flow, syncbar_task
from workflows.configs import renko_v1, renko_v2
from utilities import date_fu, project_globals as g
from data_layer import storage_adaptor, fsspec_factory, data_access, arrow_dataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
config = renko_v2.config

prefix_data = '/data/trades'

prefix_meta = f"/bars/{config['meta']['config_id']}/meta"

prefix_df = f"/bars/{config['meta']['config_id']}/df"

# Ray setup

In [76]:
ray.init(dashboard_host='0.0.0.0', dashboard_port=1111, ignore_reinit_error=True)

2021-06-09 22:02:39,233	INFO services.py:1272 -- View the Ray dashboard at http://192.168.1.66:1111


{'node_ip_address': '192.168.1.66',
 'raylet_ip_address': '192.168.1.66',
 'redis_address': '192.168.1.66:6379',
 'object_store_address': '/tmp/ray/session_2021-06-09_22-02-35_238975_40298/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-06-09_22-02-35_238975_40298/sockets/raylet',
 'webui_url': '192.168.1.66:1111',
 'session_dir': '/tmp/ray/session_2021-06-09_22-02-35_238975_40298',
 'metrics_export_port': 59387,
 'node_id': 'cadc0bca39dd8c310e36eb958df840fb2e82902be51fccd1967d419c'}

In [ ]:
ray.shutdown()

# sync bars

In [ ]:
syncbar_flow.syncbar_flow('renko_v2', on_ray='True')

(pid=41034) HMY AEM 2019-02-05 zero tick syncbar
(pid=41034) clock: HMY sync: AEM 2019-02-05
(pid=41034) HMY AEM 2019-02-08 zero tick syncbar
(pid=41033) clock: HMY sync: AEM 2019-02-04
(pid=41033) HMY AEM 2019-02-11 zero tick syncbar
(pid=41035) clock: HMY sync: AEM 2019-02-06
(pid=41036) clock: HMY sync: AEM 2019-02-07
(pid=41034) clock: HMY sync: AEM 2019-02-08
(pid=41033) clock: HMY sync: AEM 2019-02-11
(pid=41034) clock: HMY sync: AEM 2019-02-14
(pid=41033) clock: HMY sync: AEM 2019-02-15
(pid=41036) clock: HMY sync: AEM 2019-02-13
(pid=41035) clock: HMY sync: AEM 2019-02-12
(pid=41036) clock: HMY sync: AEM 2019-02-21
(pid=41034) clock: HMY sync: AEM 2019-02-19
(pid=41033) clock: HMY sync: AEM 2019-02-20
(pid=41035) clock: HMY sync: AEM 2019-02-22
(pid=41036) clock: HMY sync: AEM 2019-02-25
(pid=41035) clock: HMY sync: AEM 2019-02-28
(pid=41033) clock: HMY sync: AEM 2019-02-27
(pid=41034) clock: HMY sync: AEM 2019-02-26
(pid=41033) clock: HMY sync: AU 2019-02-05
(pid=41035) clock:

# Tick Sampler 

In [ ]:
bds = sampler_flow.run(config)

In [ ]:
n = 1
print(bds[n]['date'])
# bds[n]['ticks_df'][['price', 'price_jma']].plot_bokeh(sizing_mode="scale_height")
bds[n]['ticks_df'][['price', 'price_jma']].plot()

In [ ]:
bds[n]['ticks_df'][['price_jma']].plot()

In [ ]:
bds[n]['ticks_df'].status.value_counts() / len(bds[n]['ticks_df'])

In [ ]:
bds[n]['bars_df'][['price_low', 'price_high', 'price_close']].plot()

# Price gap fill

In [ ]:
from sample_features import stacked
# fill daily gaps
stacked_df = stacked.fill_gaps_dates(bar_dates, fill_col='price_vwap')

# stacked_df[['price_high','price_low','price_close', 'price_vwap']].plot_bokeh(sizing_mode="scale_height")

# stacked stats
stats_df = stacked.stacked_df_stats(stacked_df)


In [ ]:
stats_df.describe()